# 0. Table - EDB

## Table 1: Product

### Col 1: item

### Col 2: price

## Table 2: Product

### Col 1: item

### Col 2: country

In [1]:
productCol = ["Product","Price"]
madeinCol = ["Product", "Country"]

In [2]:
tables = ["madein", "price"]

In [3]:
tabCol = {tables[0]:madeinCol, tables[1]:productCol}

In [4]:
countries = ["germany","belgium","uk","china","usa","japan"]
prices = ["5","6","10"]

In [5]:
consDict = {"Country":countries,  "Price": prices}

In [6]:
consDict

{'Country': ['germany', 'belgium', 'uk', 'china', 'usa', 'japan'],
 'Price': ['5', '6', '10']}

# 1. From Datalog to Sql

## Datalog Programs Predicates

## $p = \{p_1,p_2,p_3\}$

### $p_1:$ good_product

- Made in Germany or Belgium

### $p_2:$ bad_product

- Not made in Germany, Belgium or UK

### $p_3:$ perfect_product

- Price < 5 and not a bad product

## Datalog Program P

In [7]:
datalog_goodProd = "good_product(Product):-madein(Product,germany).good_product(Product):-madein(Product,belgium)."

datalog_badProd = "bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk."

datalog_perfProd = "perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5."

In [34]:
datalog_goodProd

'good_product(Product):-madein(Product,germany).good_product(Product):-madein(Product,belgium).'

In [35]:
datalog_badProd

'bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk.'

In [33]:
datalog_perfProd

'perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5.'

In [8]:
import re

In [9]:
sep1 = '.'
sep2 = ':-'
sep3 = ', '
cmps = ["!=",">","<","<=",">=","=="]

In [10]:
datalog_goodProd
datalog_badProd
datalog_perfProd

'perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5.'

In [11]:
def getClauses(datalog_query):
    return [item for item in datalog_query.split(sep1) if item != '']

In [12]:
def getHeads(datalog_query):
    clauses = getClauses(datalog_query)
    return [clause.split(sep2)[0] for clause in clauses]

In [13]:
def getAllPredicates(datalog_query):
    return [head.split('(')[0] for head in getHeads(datalog_query)]

In [14]:
def getVariablesInHeads(datalog_query):
    heads = getHeads(datalog_query)
    query = []
    for head in heads:
        query += [q.strip() for q in head[head.find("(")+1:head.find(")")].split(',')]
    return query

In [15]:
def getBodies(datalog_query):
    clauses = getClauses(datalog_query)
    return [clause.split(sep2)[1] for clause in clauses]

In [224]:
getClauses(inp)

['good_product(Product):-madein(Product,germany)',
 'good_product(Product):-madein(Product,belgium)',
 'bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk',
 'perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5']

In [235]:
def getHBDict(datalog_query):
    result = {}
    clauses = getClauses(datalog_query)
    i = 1
    for clause in clauses:
        result["Clause"+str(i)] = clause.split(sep2)[0]+sep2+clause.split(sep2)[1]
        i += 1
    return result

In [236]:
def getTotalPredicates(datalog_query):
    result = []
    for body in getBodies(datalog_query):
        for term in body.split(', '):
            result.append(term)
    for head in getHeads(datalog_query):
        result.append(head)
    return result

In [237]:
def getLiteralHeadDict(datalog_query):
    bH = getHBDict(datalog_query)
    result = {}
    for key in bH.keys():
        if len(key.split(sep3))>1:
            for term in key.split(sep3):
                result[term] = bH[key]
        else:
            result[key.split(sep3)[0]] = bH[key]
    return result

In [238]:
def getVariablesInBodies(datalog_query):
    query = []
    for body in getHBDict(datalog_query).keys():
        for term in body.split(', '):
            if term.find("(") != -1:
                query += [q.strip() for q in term[term.find("(")+1:term.find(")")].split(',')]
            else:
                for cmp in cmps:
                    if term.find(cmp) != -1:
                        query.append(term.split(cmp)[0])
                        query.append(term.split(cmp)[1])
    return query

In [239]:
def getVariablesInClauses(datalog_query):
    query = []
    for body in getHBDict(datalog_query).keys():
        for term in body.split(', '):
            if term.find("(") != -1:
                query += [q.strip() for q in term[term.find("(")+1:term.find(")")].split(',')]
            else:
                for cmp in cmps:
                    if term.find(cmp) != -1:
                        query.append(term.split(cmp)[0])
                        query.append(term.split(cmp)[1])
    for body in getHBDict(datalog_query).values():
        for term in body.split(', '):
            if term.find("(") != -1:
                query += [q.strip() for q in term[term.find("(")+1:term.find(")")].split(',')]
            else:
                for cmp in cmps:
                    if term.find(cmp) != -1:
                        query.append(term.split(cmp)[0])
                        query.append(term.split(cmp)[1])
    return query

In [240]:
def getPVDict(datalog_query):
    predConsDict = {}
    i = 1
    for var in getVariablesInClauses(datalog_query):
        for ls in consDict.values():
            if var in ls:
                if var in predConsDict.keys():
                    continue
                else:
                    predConsDict[var] = 'P'+str(i)
                    i = i + 1
    return predConsDict

In [241]:
inp = datalog_goodProd+datalog_badProd+datalog_perfProd

In [242]:
inp

'good_product(Product):-madein(Product,germany).good_product(Product):-madein(Product,belgium).bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk.perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5.'

## Step 1. Replace Constant with variables and add new Predicates

In [243]:
pvDict = getPVDict(inp)

In [244]:
pvDict

{'5': 'P4', 'belgium': 'P2', 'germany': 'P1', 'uk': 'P3'}

In [245]:
newPredicates = []
for key in pvDict.keys():
    newPredicates.append(pvDict[key]+'='+'{('+key+')};')

In [246]:
newPredicates

['P1={(germany)};', 'P2={(belgium)};', 'P3={(uk)};', 'P4={(5)};']

In [247]:
bH = getHBDict(inp)

In [248]:
bH

{'Clause1': 'good_product(Product):-madein(Product,germany)',
 'Clause2': 'good_product(Product):-madein(Product,belgium)',
 'Clause3': 'bad_product(Product):-madein(Product,Country), Country!=germany, Country!=belgium, Country!=uk',
 'Clause4': 'perfect_product(Product):-good_product(Product), not bad_product(belgium), price(Product,Price), Price<5'}

In [249]:
consDict

{'Country': ['germany', 'belgium', 'uk', 'china', 'usa', 'japan'],
 'Price': ['5', '6', '10']}

In [250]:
consts = []
for key in consDict.keys():
    consts+=consDict[key]

In [252]:
newDict = {}
for key in bH.keys():
    i = 1
    clause = bH[key]
    for const in consts:     
        if clause.find(const) != -1:
            variable = "v" + str(i)
            clause = clause.replace(const,variable)
            add = ", " + pvDict[const] + "(" + variable + ")"
            temp = clause + add
            i = i + 1
    newDict[key] = temp 

In [253]:
newDict

{'Clause1': 'good_product(Product):-madein(Product,v1), P1(v1)',
 'Clause2': 'good_product(Product):-madein(Product,v1), P2(v1)',
 'Clause3': 'bad_product(Product):-madein(Product,Country), Country!=v1, Country!=v2, Country!=v3, P3(v3)',
 'Clause4': 'perfect_product(Product):-good_product(Product), not bad_product(v1), price(Product,Price), Price<v2, P4(v2)'}

## Step 2. Replace Equation

## Step 3. Combine Multiple-Clause Predicate Definitions

In [254]:
newPredStart = len(pvDict)

In [255]:
# Find duplicate predicate
predCount = {}
duplicatePred = []
for key in newDict.keys():
    a = newDict[key].split(':')[0]
    b = a.split('(')[0]
    if b in predCount:
        duplicatePred.append(b)
        predCount[b] += 1
    else:
        predCount[b] = 1

In [256]:
predCount

{'bad_product': 1, 'good_product': 2, 'perfect_product': 1}

In [257]:
i = len(pvDict) + 1
newPredicate3 = {}
for key in newDict.keys():
    a = newDict[key].split(':')[0]
    b = a.split('(')[0]
    newPredicate3[b] = ''
    
for key in newDict.keys():
    a = newDict[key].split(':')[0]
    b = a.split('(')[0]
    if predCount[b] > 1:
        newDict[key] = newDict[key].replace(b,'P'+str(i))
        newPredicate3[b] += 'P'+str(i)+' logicAnd '
        i += 1

In [258]:
newPredicate3

{'bad_product': '',
 'good_product': 'P5 logicAnd P6 logicAnd ',
 'perfect_product': ''}

In [259]:
newPredicate3Cleard = {}
for key in newPredicate3.keys():
    if newPredicate3[key] != '':
        newPredicate3Cleard[key] = newPredicate3[key].strip('logicAnd ')

In [260]:
newPredicate3Cleard

{'good_product': 'P5 logicAnd P6'}

In [261]:
newDict

{'Clause1': 'P5(Product):-madein(Product,v1), P1(v1)',
 'Clause2': 'P6(Product):-madein(Product,v1), P2(v1)',
 'Clause3': 'bad_product(Product):-madein(Product,Country), Country!=v1, Country!=v2, Country!=v3, P3(v3)',
 'Clause4': 'perfect_product(Product):-good_product(Product), not bad_product(v1), price(Product,Price), Price<v2, P4(v2)'}

In [262]:
newPredicates

['P1={(germany)};', 'P2={(belgium)};', 'P3={(uk)};', 'P4={(5)};']

## Step 4. Rename variables which appears multiple times in a clauses

### Get variables for each clause

In [265]:
# Count variables in head
for key in newDict.keys():
    head = newDict[key].split(':-')[0]
    body = newDict[key].split(':-')[1]
    bts = body.split(', ')
    bvLoc = {}
    hvs = list(set(head[head.find("(")+1:head.find(")")].split(',')))
    hvDict = {}
    i = 1
    for hv in hvs:
        hvDict[hv] = x + str(i)
        i += 1
    # replace variables in head
    for key in hvDict.keys():
        head = head.replace(key, hvDict[key])
    
    bvs = []
    # get variables in body
    for bt in bts:
        terms = bt[bt.find("(")+1:bt.find(")")].split(',')
        for term in terms:
            bv.append(term)
    bvs = list(set(bvs))
    
    # replace
    for key in hvDict.keys():
        body.replace(key, hvDict[key])

['Product']
['Product']
['Product']
['Product']
